In [1]:
import torch
from GASU_env_host import GASU
from GASU_optimizer import optimize_GASU

In [ ]:
# Episode Length: T
T = 31
state_horizon = 30
action_horizon = 1
simulation_days = T + state_horizon - action_horizon       # Total simulation days = 60
env_id = 'GASU-v0'
env = GASU(env_id, config_path="gasu_config.json")

# Preprocessing
# Step 1: Rendering essential information from environment
compressors = env.compressor_info()       # Provides the infromation compressors and its initial state in dict format
external_purchase_price = env.get_external_purchase_price()    # Provides the dict of external purchase price
external_purchase_capacity = env.get_external_purchase_capacity()    # Provides the dict of external purchase capacity

# Step 1: Initializing the optimizer by feeding the essential information from environment
optimizer = optimize_GASU(state_horizon, action_horizon, compressors)
optimizer._update_external_purchase_price(external_purchase_price)         # __Update the external purchase price in optimizer__
optimizer._update_external_purchase_capacity(external_purchase_capacity)   # __Update the external purchase capacity in optimizer__

### Simulation for 31 days (with optimal action)

In [4]:
env.reset()
flatt_state = env._get_state('flatt')  # mode = 'flatt'
opt_episode_reward = 0
simulation_episode_reward = 0
terminated = torch.tensor(False)

while terminated.item() is False:

    optimizer.update_state(flatt_state)
    objective_value = optimizer.solve()
    opt_cost = optimizer.action_horizon_cost()
    opt_action = optimizer._fetch_optimal_action_for_action_horizon()
    # feed action to environment, # tensor output
    flatt_state_tensor, reward, terminated, truncated, info = env.step(opt_action)
    flatt_state = env._get_state('flatt')
    opt_episode_reward += opt_cost
    simulation_episode_reward += reward.item()
    
print(f"Optimal Episode Reward: {opt_episode_reward}")
print(f"Final Episode Reward: {simulation_episode_reward}")

Optimal Episode Reward: 1221.8517861219732
Final Episode Reward: -1221.3104362487793
